# Natural Language Understanding
## Querying a Database

In [2]:
import nltk

*Problem*

a.		Which country is Athens in?

b.		Greece.

Table 1.1:

city_table: A table of cities, countries and populations

|City|Country|Population|
|----|-------|----------|
|athens|greece|1368|
|bangkok|thailand|1178|
|barcelona|spain|1280|
|berlin|east_germany|3481|
|birmingham|united_kingdom|1112|

SQL `	SELECT Country FROM city_table WHERE City = 'athens'` can easily get the answer

How to translate English input to SQL?

In [2]:
nltk.data.show_cfg('grammars/book_grammars/sql0.fcfg')

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
N[SEM='City FROM city_table'] -> 'cities'
IV[SEM=''] -> 'are'
A[SEM=''] -> 'located'
P[SEM=''] -> 'in'


In [3]:
from nltk import load_parser
cp = load_parser('grammars/book_grammars/sql0.fcfg')
query = 'What cities are located in China'
trees = list(cp.parse(query.split()))
answer = trees[0].label()['SEM']
answer = [s for s in answer if s]
q = ' '.join(answer)
print(q)

SELECT City FROM city_table WHERE Country="china"


In [10]:
from nltk.sem import chat80
rows = chat80.sql_query('corpora/city_database/city.db', q)
for r in rows: 
    print(r[0], end=" ")

canton chungking dairen harbin kowloon mukden peking shanghai sian tientsin 

## Natural Language, Semantics and Logic

Broadly speaking, logic-based approaches to natural language semantics focus on those aspects of natural language which guide our judgments of consistency and inconsistency. The syntax of a logical language is designed to make these features formally explicit. As a result, determining properties like consistency can often be reduced to symbolic manipulation, that is, to a task that can be carried out by a computer. In order to pursue this approach, we first want to develop a technique for representing a possible situation. We do this in terms of something that logicians call a model.

A **model** for a set W of sentences is a formal representation of a situation in which all the sentences in W are true. The usual way of representing models involves set theory. The domain D of discourse (all the entities we currently care about) is a set of individuals, while relations are treated as sets built up from D. Let's look at a concrete example. Our domain D will consist of three children, Stefan, Klaus and Evi, represented respectively as s, k and e. We write this as D = {s, k, e}. The expression boy denotes the set consisting of Stefan and Klaus, the expression girl denotes the set consisting of Evi, and the expression is running denotes the set consisting of Stefan and Evi. 1.2 is a graphical rendering of the model.

Figure 1.2
![](http://www.nltk.org/images/model_kids.png)

# Propositional Logic
A logical language is designed to make reasoning formally explicit. As a result, it can capture aspects of natural language which determine whether a set of sentences is consistent. As part of this approach, we need to develop logical representations of a sentence φ which formally capture the **truth-conditions** of φ. 

**Propositional logic** allows us to represent just those parts of linguistic structure which correspond to certain sentential connectives. We have just looked at and. Other such connectives are not, or and if..., then.... In the formalization of propositional logic, the counterparts of such connectives are sometimes called **boolean operators**. The basic expressions of propositional logic are **propositional symbols**, often written as P, Q, R, etc. There are varying conventions for representing boolean operators. Since we will be focusing on ways of exploring logic within NLTK, we will stick to the following ASCII versions of the operators:

In [12]:
nltk.boolean_ops()

negation       	-
conjunction    	&
disjunction    	|
implication    	->
equivalence    	<->


From the propositional symbols and the boolean operators we can build an infinite set of **well formed formulas** (or just formulas, for short) of propositional logic. First, every propositional letter is a formula. Then if φ is a formula, so is -φ. And if φ and ψ are formulas, then so are (φ & ψ) (φ | ψ) (φ -> ψ) (φ <-> ψ).

Table 2.1:

Truth conditions for the Boolean Operators in Propositional Logic.

|Boolean Operator|Truth Conditions|
|----------------|----------------|
|negation (it is not the case that ...)	-φ is true in s|iff	φ is false in s|
|conjunction (and)	(φ & ψ) is true in s|iff	φ is true in s and ψ is true in s|
|disjunction (or)	(φ &#124; ψ) is true in s|iff	φ is true in s or ψ is true in s|
|implication (if ..., then ...)	(φ -> ψ) is true in s|iff	φ is false in s or ψ is true in s|
|equivalence (if and only if)	(φ <-> ψ) is true in s|iff	φ and ψ are both true in s or both false in s|

In [3]:
read_expr = nltk.sem.Expression.fromstring
read_expr('-(P & Q)')

<NegatedExpression -(P & Q)>

In [15]:
read_expr('P & Q')

<AndExpression (P & Q)>

In [16]:
read_expr('P | (R -> Q)')

<OrExpression (P | (R -> Q))>

In [19]:
read_expr('P <-> -- P')

<IffExpression (P <-> --P)>

Logical proofs can be carried out with NLTK's inference module, for example via an interface to the third-party theorem prover Prover9. The inputs to the inference mechanism first have to be converted into logical expressions.

In [4]:
lp = nltk.sem.Expression.fromstring
SnF = read_expr('SnF')
NotFnS = read_expr('-FnS')
R = read_expr('SnF -> -FnS')
prover = nltk.Prover9()
prover.prove(NotFnS,[SnF,R])

True

In [5]:
val = nltk.Valuation([('P',True),('Q',True),('R',False)])

In [6]:
val['P']

True

In [9]:
dom = set()
g = nltk.Assignment(dom)

m = nltk.Model(dom,val)
print(m.evaluate('(P & Q)',g))
print(m.evaluate('-(P & Q)',g))
print(m.evaluate('(P & R)',g))
print(m.evaluate('(P | R)',g))

True
False
False
True


# First-Prder Logic
## Syntax
First-order logic keeps all the boolean operators of Propositional Logic. But it adds some important new mechanisms. To start with, propositions are analyzed into predicates and arguments, which takes us a step closer to the structure of natural languages. The standard construction rules for first-order logic recognize terms such as individual variables and individual constants, and predicates which take differing numbers of arguments. For example, *Angus walks* might be formalized as *walk(angus)* and *Angus sees Bertie* as *see(angus, bertie)*. We will call *walk* a **unary predicate**, and *see* a **binary predicate**

It is often helpful to inspect the syntactic structure of expressions of first-order logic, and the usual way of doing this is to assign types to expressions. Following the tradition of Montague grammar, we will use two **basic types**: e is the type of entities, while t is the type of formulas, i.e., expressions which have truth values. Given these two basic types, we can form **complex types** for function expressions. That is, given any types σ and τ, 〈σ, τ〉 is a complex type corresponding to functions from 'σ things' to 'τ things'. For example, 〈e, t〉 is the type of expressions from entities to truth values, namely unary predicates.

In [12]:
read_expr = nltk.sem.Expression.fromstring
expr = read_expr('walk(angus)',type_check=True)
print(expr.argument)
print(expr.argument.type)
print(expr.function)
print(expr.function.type)

angus
e
walk
<e,?>


In [13]:
sig = {'walk':'<e,t>'}
expr = read_expr('walk(angus)',signature=sig)
expr.function.type

e

Each instance expr of this class comes with a method `free()` which returns the set of variables that are free in expr.

In [6]:
read_expr = nltk.sem.Expression.fromstring
print(read_expr('dog(cyril)').free())
print(read_expr('dog(x)').free())
print(read_expr('own(angus,cyril)').free())
print(read_expr('exists x. dog(x)').free())
print(read_expr('((some x. walk(x)) -> sing(x))').free())
print(read_expr('exists x. own(y,x)').free())

set()
{Variable('x')}
set()
set()
{Variable('x')}
{Variable('y')}


## First Order Theorem Proving
The general case in theorem proving is to determine whether a formula that we want to prove (a proof goal) can be derived by a finite sequence of inference steps from a list of assumed formulas. We write this as S ⊢ g, where S is a (possibly empty) list of assumptions, and g is a proof goal. We will illustrate this with NLTK's interface to the theorem prover Prover9. First, we parse the required proof goal and the two assumptions. Then we create a Prover9 instance, and call its prove() method on the goal, given the list of assumptions.
```
Sylvania is to the north of Freedonia. Therefore, Freedonia is not to the north of Sylvania
```

In [8]:
NotFnS = read_expr('-north_of(f,s)')
SnF = read_expr('north_of(s,f)')
R = read_expr('all x. all y. (north_of(x,y) -> -north_of(y,x))')
prover = nltk.Prover9()
prover.prove(NotFnS,[SnF,R])

True

In [9]:
FnS = read_expr('north_of(f,s)')
prover.prove(FnS,[SnF,R])

False

## Summarizing the Language of First Order Logic
We'll take this opportunity to restate our earlier syntactic rules for propositional logic and add the formation rules for quantifiers; together, these give us the syntax of first order logic. In addition, we make explicit the types of the expressions involved. We'll adopt the convention that 〈en, t〉 is the type of a predicate which combines with n arguments of type e to yield an expression of type t. In this case, we say that n is the arity of the predicate.

1. If P is a predicate of type 〈en, t〉, and α1, ... αn are terms of type e, then P(α1, ... αn) is of type t.
2. If α and β are both of type e, then (α = β) and (α != β) are of type t.
3. If φ is of type t, then so is -φ.
4. If φ and ψ are of type t, then so are (φ & ψ), (φ | ψ), (φ -> ψ) and (φ <-> ψ).
5. If φ is of type t, and x is a variable of type e, then exists x.φ and  all x.φ are of type t.


Table 3.1:

Summary of new logical relations and operators required for First Order Logic, together with two useful methods of the Expression class.

|Example|Description|
|-------|-----------|
|=|equality|
|!=|inequality|
|exists|existential quantifier|
|all|universal quantifier|
|e.free()|show free variables of e|
|e.simplify()|carry out β-reduction on e|

## Truth in Model
Given a first-order logic language L, a model M for L is a pair 〈D, Val〉, where D is an nonempty set called the domain of the model, and Val is a function called the valuation function which assigns values from D to expressions of L as follows:

- For every individual constant c in L, Val(c) is an element of D.
- For every predicate symbol P of arity n ≥ 0, Val(P) is a function from Dn to {True, False}. (If the arity of P is 0, then Val(P) is simply a truth value, the P is regarded as a propositional symbol.)

According to (ii), if P is of arity 2, then Val(P) will be a function f from pairs of elements of D to {True, False}. In the models we shall build in NLTK, we'll adopt a more convenient alternative, in which Val(P) is a set S of pairs, defined as follows:
```
(23)		S = {s | f(s) = True}
```
Such an f is called the characteristic function of S.

In [10]:
dom = {'b','o','c'}

We will use the utility function Valuation.fromstring() to convert a list of strings of the form symbol => value into a Valuation object.

In [11]:
v = """
bertie => b
olive => o
cyril => c
boy => {b}
girl => {o}
dog => {c}
walk => {o, c}
see => {(b, o), (c, b), (o, c)}
"""
val = nltk.Valuation.fromstring(v)
print(val)

{'bertie': 'b',
 'boy': {('b',)},
 'cyril': 'c',
 'dog': {('c',)},
 'girl': {('o',)},
 'olive': 'o',
 'see': {('o', 'c'), ('b', 'o'), ('c', 'b')},
 'walk': {('c',), ('o',)}}


## Individual Variables and Assignments
In our models, the counterpart of a context of use is a variable assignment. This is a mapping from individual variables to entities in the domain. Assignments are created using the Assignment constructor, which also takes the model's domain of discourse as a parameter. We are not required to actually enter any bindings, but if we do, they are in a (variable, value) format similar to what we saw earlier for valuations.

In [13]:
g = nltk.Assignment(dom, [('x', 'o'), ('y', 'c')])
g

{'x': 'o', 'y': 'c'}

In [14]:
print(g)

g[c/y][o/x]


In [15]:
m = nltk.Model(dom,val)
m.evaluate('see(olive,y)',g)

True

In [17]:
m.evaluate('see(y, x)', g)

False

The method purge() clears all bindings from an assignment.

In [18]:
g.purge()
g

{}

In [19]:
m.evaluate('see(olive, y)', g)

'Undefined'

In [20]:
m.evaluate('see(bertie, olive) & boy(bertie) & -walk(bertie)', g)

True

## Quantification
One of the crucial insights of modern logic is that the notion of variable satisfaction can be used to provide an interpretation to quantified formulas. Let's use (24) as an example.
```
(24)		exists x.(girl(x) & walk(x))
```

In [21]:
m.evaluate('exists x. (girl(x) & walk(x))',g)

True

One useful tool offered by NLTK is the satisfiers() method. This returns a set of all the individuals that satisfy an open formula. The method parameters are a parsed formula, a variable, and an assignment.

In [24]:
fmla1 = read_expr('girl(x) | boy(x)')
m.satisfiers(fmla1,'x',g)

{'b', 'o'}

In [25]:
fmla2 = read_expr('girl(x) -> walk(x)')
m.satisfiers(fmla2,'x',g)

{'b', 'c', 'o'}

In [26]:
fmla3 = read_expr('walk(x) -> girl(x)')
m.satisfiers(fmla3,'x',g)

{'b', 'o'}

In [27]:
m.evaluate('all x.(girl(x) -> walk(x))', g)

True

## Quantifier Scope Ambiguity
What happens when we want to give a formal representation of a sentence with two quantifiers, such as the following?
```
(26)		Everybody admires someone.
```
There are (at least) two ways of expressing (26) in first-order logic:
```
(27)		
a.		all x.(person(x) -> exists y.(person(y) & admire(x,y)))

b.		exists y.(person(y) & all x.(person(x) -> admire(x,y)))
```


In [28]:
v2 = """
bruce => b
elspeth => e
julia => j
matthew => m
person => {b, e, j, m}
admire => {(j, b), (b, b), (m, e), (e, m)}
"""
val2 = nltk.Valuation.fromstring(v2)

dom2 = val2.domain
m2 = nltk.Model(dom2,val2)
g2 = nltk.Assignment(dom2)
fmla4 = read_expr('(person(x) -> exists y.(person(y) & admire(x, y)))')
m2.satisfiers(fmla4, 'x', g2)

{'b', 'e', 'j', 'm'}

In [33]:
fmla5 = read_expr('(person(y) & all x.(person(x) -> admire(x, y)))')
m2.satisfiers(fmla5, 'y', g2)

set()

In [34]:
fmla6 = read_expr('(person(y) & all x.((x = bruce | x = julia) -> admire(x, y)))')
m2.satisfiers(fmla6, 'y', g2)

{'b'}

## Model Building
We invoke the Mace4 model builder by creating an instance of Mace() and calling its build_model() method,

In [35]:
a3 = read_expr('exists x. (man(x) & walks(x))')
c1 = read_expr('mortal(socrates)')
c2 = read_expr('-mortal(socrates)')
mb = nltk.Mace(5)
print(mb.build_model(None,[a3,c1]))

True


In [37]:
print(mb.build_model(None, [a3, c2]))

True


In [38]:
print(mb.build_model(None, [c1, c2]))

False


We can also use the model builder as an adjunct to the theorem prover. Let's suppose we are trying to prove S ⊢ g, i.e. that g is logically derivable from assumptions S = [s1, s2, ..., sn]. We can feed this same input to Mace4, and the model builder will try to find a counterexample, that is, to show that g does not follow from S. So, given this input, Mace4 will try to find a model for the set S together with the negation of g, namely the list S' =
[s1, s2, ..., sn, -g]. If g fails to follow from S, then Mace4 may well return with a counterexample faster than Prover9 concludes that it cannot find the required proof. Conversely, if g is provable from S, Mace4 may take a long time unsuccessfully trying to find a countermodel, and will eventually give up.

In [39]:
a4 = read_expr('exists y. (woman(y) & all x. (man(x) -> love(x,y)))')
a5 = read_expr('man(adam)')
a6 = read_expr('woman(eve)')
g = read_expr('love(adam,eve)')
mc = nltk.MaceCommand(g, assumptions=[a4, a5, a6])
mc.build_model()

True

In [40]:
print(mc.valuation)

{'C1': 'b',
 'adam': 'a',
 'eve': 'a',
 'love': {('a', 'b')},
 'man': {('a',)},
 'woman': {('a',), ('b',)}}


The general form of this valuation should be familiar to you: it contains some individual constants and predicates, each with an appropriate kind of value. What might be puzzling is the C1. This is a "skolem constant" that the model builder introduces as a representative of the existential quantifier. That is, when the model builder encountered the exists y part of a4 above, it knew that there is some individual b in the domain which satisfies the open formula in the body of a4. However, it doesn't know whether b is also the denotation of an individual constant anywhere else in its input, so it makes up a new name for b on the fly, namely  C1. Now, since our premises said nothing about the individual constants adam and eve, the model builder has decided there is no reason to treat them as denoting different entities, and they both get mapped to a. Moreover, we didn't specify that man and woman denote disjoint sets, so the model builder lets their denotations overlap. This illustrates quite dramatically the implicit knowledge that we bring to bear in interpreting our scenario, but which the model builder knows nothing about. So let's add a new assumption which makes the sets of men and women disjoint. The model builder still produces a countermodel, but this time it is more in accord with our intuitions about the situation:

In [41]:
a7 = read_expr('all x. (man(x) -> -woman(x))')
g = read_expr('love(adam,eve)')
mc = nltk.MaceCommand(g, assumptions=[a4, a5, a6, a7])
mc.build_model()

True

In [42]:
print(mc.valuation)

{'C1': 'c',
 'adam': 'a',
 'eve': 'b',
 'love': {('a', 'c')},
 'man': {('a',)},
 'woman': {('c',), ('b',)}}


# Semantics of English Sentences
## Compositional Semantics in Feature-Based Grammar
**Principle of Compositionality**: The meaning of a whole is a function of the meanings of the parts and of the way they are syntactically combined.


 (29) illustrates a first approximation to the kind of analyses we would like to build.

(29)		![](http://www.nltk.org/book/tree_images/ch10-tree-1.png)


To complete the grammar is very straightforward; all we require are the rules shown below.
```
VP[SEM=?v] -> IV[SEM=?v]
NP[SEM=<cyril>] -> 'Cyril'
IV[SEM=<\x.bark(x)>] -> 'barks'
```

## The λ-Calculus
We illustrated this with (31), which we glossed as "the set of all w such that w is an element of V (the vocabulary) and w has property P".
```
(31)		{w | w ∈ V & P(w)}
```
It turns out to be extremely useful to add something to first-order logic that will achieve the same effect. We do this with the λ operator (pronounced "lambda"). The λ counterpart to (31) is (32). (Since we are not trying to do set theory here, we just treat V as a unary predicate.)
```
(32)		λw. (V(w) ∧ P(w))
```

λ is a binding operator, just as the first-order logic quantifiers are. If we have an open formula such as (33a), then we can bind the variable x with the λ operator, as shown in (33b). The corresponding NLTK representation is given in (33c).
```
(33)		
a.		(walk(x) ∧ chew_gum(x))

b.		λx.(walk(x) ∧ chew_gum(x))

c.		\x.(walk(x) & chew_gum(x))
```
Remember that \ is a special character in Python strings. We could escape it (with another \), or else use "raw strings" 

In [43]:
read_expr = nltk.sem.Expression.fromstring
expr = read_expr(r'\x.(walk(x) & chew_gum(x))')
expr

<LambdaExpression \x.(walk(x) & chew_gum(x))>

In [44]:
expr.free()

set()

In [45]:
print(read_expr(r'\x.(walk(x) & chew_gum(y))'))

\x.(walk(x) & chew_gum(y))


We have a special name for the result of binding the variables in an expression: **λ abstraction**. When you first encounter λ-abstracts, it can be hard to get an intuitive sense of their meaning. A couple of English glosses for (33b) are: "be an x such that x walks and x chews gum" or "have the property of walking and chewing gum". It has often been suggested that λ-abstracts are good representations for verb phrases (or subjectless clauses), particularly when these occur as arguments in their own right. This is illustrated in (34a) and its translation (34b).
```
(34)		
a.		To walk and chew-gum is hard

b.		hard(\x.(walk(x) & chew_gum(x)))
```

So the general picture is this: given an open formula φ with free variable x, abstracting over x yields a property expression λx.φ — the property of being an x such that φ. Here's a more official version of how abstracts are built:
```
(35)		If α is of type τ, and x is a variable of type e, then \x.α is of type 〈e, τ〉.
```

(34b) illustrated a case where we say something about a property, namely that it is hard. But what we usually do with properties is attribute them to individuals. And in fact if φ is an open formula, then the abstract λx.φ can be used as a unary predicate. In (36), (33b) is predicated of the term gerald.
```
(36)		\x.(walk(x) & chew_gum(x)) (gerald)
```
Now (36) says that Gerald has the property of walking and chewing gum, which has the same meaning as (37).
```
(37)		(walk(gerald) & chew_gum(gerald))
```

We'll use α[β/x] as notation for the operation of replacing all free occurrences of x in α by the expression β. So:
```
(walk(x) & chew_gum(x))[gerald/x]
```
is the same expression as (37). The "reduction" of (36) to (37) is an extremely useful operation in simplifying semantic representations, and we shall use it a lot in the rest of this chapter. The operation is often called β-reduction. In order for it to be semantically justified, we want it to hold that λx. α(β) has the same semantic values as α[β/x].

 In order to carry of β-reduction of expressions in NLTK, we can call the simplify() method

In [46]:
expr = read_expr(r'\x.(walk(x) & chew_gum(x))(gerald)')
print(expr)

\x.(walk(x) & chew_gum(x))(gerald)


In [47]:
print(expr.simplify())

(walk(gerald) & chew_gum(gerald))


In [48]:
print(read_expr(r'\x.\y.(dog(x) & own(y, x))(cyril)').simplify())
print(read_expr(r'\x y.(dog(x) & own(y, x))(cyril, angus)').simplify())

\y.(dog(cyril) & own(y,cyril))
(dog(cyril) & own(angus,cyril))


All our λ abstracts so far have involved the familiar first order variables: x, y and so on — variables of type e. But suppose we want to treat one abstract, say \x.walk(x) as the argument of another λ abstract? We might try this:
```
\y.y(angus)(\x.walk(x))
```
But since the variable y is stipulated to be of type e, \y.y(angus) only applies to arguments of type e while \x.walk(x) is of type 〈e, t〉! Instead, we need to allow abstraction over variables of higher type. Let's use P and Q as variables of type 〈e, t〉, and then we can have an abstract such as \P.P(angus). Since P is of type 〈e, t〉, the whole abstract is of type 〈〈e, t〉, t〉. Then \P.P(angus)(\x.walk(x)) is legal, and can be simplified via β-reduction to  \x.walk(x)(angus) and then again to walk(angus)

When carrying out β-reduction, some care has to be taken with variables. Consider, for example, the λ terms (39a) and (39b), which differ only in the identity of a free variable.
```
(39)		
a.		\y.see(y, x)

b.		\y.see(y, z)
```

Suppose now that we apply the λ-term \P.exists x.P(x) to each of these terms:
```
(40)		
a.		\P.exists x.P(x)(\y.see(y, x))

b.		\P.exists x.P(x)(\y.see(y, z))
```

We pointed out earlier that the results of the application should be semantically equivalent. But if we let the free variable x in (39a) fall inside the scope of the existential quantifier in (40a), then after reduction, the results will be different:
```
(41)		
a.		exists x.see(x, x)

b.		exists x.see(x, z)
```


Given any variable-binding expression (involving ∀, ∃ or λ), the name chosen for the bound variable is completely arbitrary. For example, exists x.P(x) and exists y.P(y) are equivalent; they are called **α equivalents**, or **alphabetic variants**. The process of relabeling bound variables is known as **α-conversion**. When we test for equality of VariableBinderExpressions in the logic module (i.e., using ==), we are in fact testing for α-equivalence:

In [49]:
expr1 = read_expr('exists x.P(x)')
print(expr1)

exists x.P(x)


In [50]:
expr2 = expr1.alpha_convert(nltk.sem.Variable('z'))
print(expr2)

exists z.P(z)


In [51]:
expr1 == expr2

True

When β-reduction is carried out on an application f(a), we check whether there are free variables in a which also occur as bound variables in any subterms of f. Suppose, as in the example discussed above, that x is free in a, and that f contains the subterm exists x.P(x). In this case, we produce an alphabetic variant of exists x.P(x), say, exists z1.P(z1), and then carry on with the reduction. This relabeling is carried out automatically by the β-reduction code in  logic, and the results can be seen in the following example.

In [52]:
expr3 = read_expr('\P.(exists x.P(x))(\y.see(y, x))')
print(expr3)

(\P.exists x.P(x))(\y.see(y,x))


In [53]:
print(expr3.simplify())

exists z1.see(z1,x)


## Quantified NPs